In [4]:
# Calcule la loi du processus X a priori à partir du signal d'origine X
def calc_probaprio(X, cl1, cl2):
    
    # Paramètres de l'estimateur de p = E(X1)
    n = 0
    
    for x in X:
        if x == cl1:
            n += 1
    
    p1 = n/len(X)
    p2 = 1 - p1
    
    return [p1, p2]

In [5]:
from scipy.stats import norm

# Classe les éléments du signal bruité suivant le critère du MAP
def MAP_MPM2(Y, cl1, cl2, p1, p2, m1, sig1, m2, sig2):
    
    S = []
    
    for y in Y:
        
        if p1*norm.pdf(y, loc=m1, scale=sig1) > p2*norm.pdf(y, loc=m2, scale=sig2):
            S.append(cl1)
        else:
            S.append(cl2)
    
    return S

In [11]:
from premiere_idee_MV import *

M1 = [120, 127, 127, 127, 127]
M2 = [130, 127, 128, 128, 128]
SIG1 = [1, 1, 1, 0.1, 2]
SIG2 = [2, 5, 1, 0.1, 3]

# Chargement des données
X = np.load("signal.npy")

# Récupération des classes de X
cl1, cl2 = np.unique(X)

# Récupération de la loi de X à priori
[p1, p2] = calc_probaprio(X, cl1, cl2)

# Construction de l'axe des abscisses
abscisse = []
for i in range(0, len(X)):
    abscisse.append(i)

for k in range(0, len(M1)):
    # Bruitage de l'échantillon
    Y = bruit_gauss2(X, cl1, cl2, M1[k], SIG1[k], M2[k], SIG2[k])
    
    # Classification
    S = MAP_MPM2(Y, cl1, cl2, p1, p2, M1[k], SIG1[k], M2[k], SIG2[k])
    
    # Plot
    plt.plot(abscisse, X, 'r-')
    plt.plot(abscisse, Y, 'g.')
    plt.plot(abscisse, S, 'b.')
    plt.title("Taux d'erreur moyen (LR): " + str(erreur_moyenne_MAP(100, p1, p2, M1[k], SIG1[k], M2[k], SIG2[k], forceX=X)))
    plt.show()

ModuleNotFoundError: No module named 'premiere_idee_MV'

In [12]:
import numpy as np

# Simule un signal de taille n composantes indépendantes à valeurs dans {cl1, cl2} avec proba {p1, p2}
def X_simu(n, cl1, cl2, p1, p2):
    
    X = []
    
    for i in range(0, n):
        u = np.random.rand()
        
        if u < p1:
            X.append(cl1)
        else:
            X.append(cl2)
    
    return X

In [ ]:
import premiere_idee_MV

# Calcul moyen du taux d'erreur
def erreur_moyenne_MAP(T, p1, p2, m1, sig1, m2, sig2, toPlot=False, forceX=np.load("signal.npy")):
    
    # Somme du calul de la moyenne
    tau = 0
    
    # Calcul de l'erreur partielle pour le plot
    erreur_partielle = []
    abscisse = []    
    
    # Chargement des données
    X = forceX

    # Récupération des classes
    cl1, cl2 = np.unique(X)
        
    for k in range(0, T):

        # Bruitage du signal
        Y = bruit_gauss2(X, cl1, cl2, m1, sig1, m2, sig2)

        # Segmentation
        S = MAP_MPM2(Y, cl1, cl2, p1, p2, m1, sig1, m2, sig2)

        # Construction de la somme
        tau += taux_erreur(X, S)
        
        # Ajout du taux d'erreur partiel
        erreur_partielle.append(tau/(k+1))
        abscisse.append(k+1)
    
    if toPlot:
        # Plot de l'erreur partielle
        plt.plot(abscisse, erreur_partielle)
        plt.title("Taux d'erreur cumulé ((" + str(m1) + ","+ str(sig1) +") -" + " (" + str(m2) + ","+ str(sig2) +"))")
        plt.show()
    
    return tau/T

# Comparaison des deux méthodes
def comparaison(n, cl1, cl2, P1=[0.1, 0.2, 0.3, 0.4, 0.5]):
    
    # Construction de l'axe des abscisses
    abscisse = []
    for i in range(0, n):
        abscisse.append(i)
        
    M1 = [120, 127, 127, 127, 127]
    M2 = [130, 127, 128, 128, 128]
    SIG1 = [1, 1, 1, 0.1, 2]
    SIG2 = [2, 5, 1, 0.1, 3]
    
    for k in range(0, len(M1)):
        
        for p1 in P1:
            
            # Génération du signal
            X = X_simu(n, cl1, cl2, p1, 1-p1)
            # Bruitage
            Y = bruit_gauss2(X, cl1, cl2, M1[k], SIG1[k], M2[k], SIG2[k])
            
            # Segmentation selon le max de vraisemblance
            S_MV = classif_gauss2(Y, cl1, cl2, M1[k], SIG1[k], M2[k], SIG2[k])
            # Segmentation selon la méthode bayésienne
            S_MAP = MAP_MPM2(Y, cl1, cl2, p1, p2, M1[k], SIG1[k], M2[k], SIG2[k])
            
            # Plot
            plt.plot(abscisse, X, 'r-')
            plt.plot(abscisse, Y, 'g.')
            plt.plot(abscisse, S_MV, 'y.')
            plt.plot(abscisse, S_MAP, 'b.')
            
            plt.title("Erreur moyenne : MV - " + str(
                erreur_moyenne_MV(100, M1[k], SIG1[k], M2[k], SIG2[k], forceX=X)) + " | MAP - " + str(
                    erreur_moyenne_MAP(100, p1, p2, M1[k], SIG1[k], M2[k], SIG2[k], forceX=X)))
            
            plt.show()